Initital setup

In [ ]:
from PIL import Image
import numpy as np
import logging as log
import time
log.basicConfig(
    format="%(asctime)s %(levelname)s: %(message)s",
    datefmt="%Y-%m-%dT%H:%M:%SZ",
    level=log.ERROR
)
EXTRA_DIFFUSE = True

Utility Functions

In [ ]:
def removeDuplicates(array):
    '''
    Removes duplicates from the given array
    '''
    newArr = []
    for i in array:
        if i not in newArr:
            newArr.append(i)
    return newArr
def printMat(mat):
    '''
    Print matrix
    '''
    for m in mat:
        print(m)
def findLoc(mat,x):
    '''
    find the x and y coordinates of the given pixel
    '''
    coordinates = np.where(mat==x)
    return [coordinates[0][0],coordinates[1][0]]

Reading image and getting RGB values

In [ ]:
def readRGB(filename,resize=False):
    '''
    Read image and return RGB representation
    '''
    img = Image.open(filename)
    if resize:
        size = 16,16
        img = img.resize(size,Image.Resampling.LANCZOS)
        img= img.convert('RGB')
    numpydata = np.asarray(img)
    R=[]
    G=[]
    B=[]
    A=[]
    for row in numpydata:
        for pixel in row:
            R.append(pixel[0])
            G.append(pixel[1])
            B.append(pixel[2])
            try:
                A.append(pixel[3])
            except IndexError:
                A.append(255)
                pass
    return (R,G,B,A,img.size)

Key Matrix Generation

In [ ]:
R,G,B,A,s=readRGB('key.png',True)
R=removeDuplicates(R)
G=removeDuplicates(G)
B=removeDuplicates(B)
R.extend([x for x in range(256) if x not in R])
RkeyMatrix = np.reshape(R, (16,16))
G.extend([x for x in range(256) if x not in G])
GkeyMatrix = np.reshape(G, (16,16))
B.extend([x for x in range(256) if x not in B])
BkeyMatrix = np.reshape(B, (16,16))
log.debug('R key matrix:')
log.debug(RkeyMatrix)
log.debug('G key matrix:')
log.debug(GkeyMatrix)
log.debug('B key matrix:')
log.debug(BkeyMatrix)
F = removeDuplicates([(int(R[i])+int(G[i])+int(B[i]))%256 for i in range(256)])
# for i in range(256):
#     s = 
#     if s not in F: 
#         F.append(s)
F.extend([x for x in range(256) if x not in F])
FinalKeyMatrix = np.reshape(F, (16,16))

Encryption or Decryption of two pixels

In [ ]:
def encryptDecryptPixel(encryptResult,i,halfLength,colorKeyMatrix,color,isEncryption=True):
    '''
    Encrypt/Decrypt the pixel data based on the Key Matrix
    '''
    floc = findLoc(colorKeyMatrix,color[i])
    sloc = findLoc(colorKeyMatrix,color[halfLength+i])
    if isEncryption:
        if EXTRA_DIFFUSE:
            floc[0]=(floc[0]+9)%16
            floc[1]=(floc[1]+9)%16
            sloc[0]=(sloc[0]+9)%16
            sloc[1]=(sloc[1]+9)%16
        if floc[0] == sloc[0]:
            encryptResult[i]=colorKeyMatrix[floc[0],(floc[1]+1)%16]
            encryptResult[halfLength+i] = colorKeyMatrix[sloc[0],(sloc[1]+1)%16]
        elif floc[1] == sloc[1]:
            encryptResult[i]=colorKeyMatrix[(floc[0]+1)%16,floc[1]]
            encryptResult[halfLength+i] = colorKeyMatrix[(sloc[0]+1)%16,sloc[1]]
        else:
            encryptResult[i]=colorKeyMatrix[floc[0],sloc[1]]
            encryptResult[halfLength+i] = colorKeyMatrix[sloc[0],floc[1]]
    else:
        if EXTRA_DIFFUSE:
            floc[0]=(floc[0]+7)%16
            floc[1]=(floc[1]+7)%16
            sloc[0]=(sloc[0]+7)%16
            sloc[1]=(sloc[1]+7)%16
        if floc[0] == sloc[0]:
            encryptResult[i]=colorKeyMatrix[floc[0],(floc[1]+15)%16]
            encryptResult[halfLength+i] = colorKeyMatrix[sloc[0],(sloc[1]+15)%16]
        elif floc[1] == sloc[1]:
            encryptResult[i]=colorKeyMatrix[(floc[0]+15)%16,floc[1]]
            encryptResult[halfLength+i] = colorKeyMatrix[(sloc[0]+15)%16,sloc[1]]
        else:
            encryptResult[i]=colorKeyMatrix[floc[0],sloc[1]]
            encryptResult[halfLength+i] = colorKeyMatrix[sloc[0],floc[1]]


Encryption Function

In [ ]:
def encrypt():
    '''
    Accept image to be encrypted and return the encrypted image based on the key
    '''
    inp = input("Enter image file name to encrypt= ")
    R,G,B,A,sizes=readRGB(inp,False)
    #R,G,B,A,sizes=readRGB('download.png',False)
    log.info(sizes)
    encryptedR=np.full(len(R),-1)
    encryptedG=np.full(len(G),-1)
    encryptedB=np.full(len(B),-1)
    log.info(encryptedR.shape)
    halfLength = len(R)//2
    for i in range(halfLength):
    #   encryptDecryptPixel(encryptedR,i,halfLength,BkeyMatrix,R)
        encryptDecryptPixel(encryptedR,i,halfLength,FinalKeyMatrix,R)
        encryptDecryptPixel(encryptedG,i,halfLength,FinalKeyMatrix,G)
        encryptDecryptPixel(encryptedB,i,halfLength,FinalKeyMatrix,B)
    #    encryptDecryptPixel(encryptedB,i,halfLength,GkeyMatrix,B)
    fullImage1D=[]
    log.debug(f'encryption {R=}')
    log.debug(f'{encryptedR=}')
    log.debug(f'encryption {G=}')
    log.debug(f'encryption {B=}')
    log.debug(f'encryption {A=}')
    for i in range(len(encryptedR)):
        fullImage1D.append([encryptedR[i],encryptedG[i],encryptedB[i],A[i]])
    # print(fullImage1D)
    x,y,z=sizes[0],sizes[1],4
    imageArr = np.reshape(fullImage1D,(y,x,z))
    # print(f'{imageArr=}')
    image = Image.fromarray(imageArr.astype('uint8'),'RGBA')
    # print(np.asarray(image))
    image.save('enc.png')

Decryption Function

In [ ]:
def decrypt():
    '''
    Accept image to be decrypted and return the decrypted image based on the key
    '''
    inp = input("Enter image file name to decrypt= ")
    R,G,B,A,sizes=readRGB(inp,False)
    # R,G,B,A,sizes=readRGB('enc.png',False)
    decryptedR=np.full(len(R),-1)
    decryptedG=np.full(len(G),-1)
    decryptedB=np.full(len(B),-1)
    halfLength = len(R)//2
    for i in range(halfLength):
        #encryptDecryptPixel(decryptedR,i,halfLength,BkeyMatrix,R,False)
        encryptDecryptPixel(decryptedR,i,halfLength,FinalKeyMatrix,R,False)
        encryptDecryptPixel(decryptedG,i,halfLength,FinalKeyMatrix,G,False)
        encryptDecryptPixel(decryptedB,i,halfLength,FinalKeyMatrix,B,False)
        #encryptDecryptPixel(decryptedB,i,halfLength,GkeyMatrix,B,False)
    fullImage1D=[]
    log.debug(f'decryption {R=}')
    log.debug(f'{decryptedR=}')
    log.debug(f'decryption {G=}')
    log.debug(f'decryption {B=}')
    log.debug(f'decryption {A=}')
    for i in range(len(decryptedR)):
        fullImage1D.append([decryptedR[i],decryptedG[i],decryptedB[i],A[i]])
    # print("fullImage1D:",len(fullImage1D))
    x,y,z=sizes[0],sizes[1],4
    imageArr = np.reshape(fullImage1D,(y,x,z))
    image = Image.fromarray(imageArr.astype('uint8'),'RGBA')
    image.save('original.png')

Driver code

In [ ]:
encrypt()
print('Encrypt successful')
time.sleep(2)
decrypt()
print('Decrypt successful')